MY logic 

In [25]:
import openpyxl
from openpyxl.styles import Border, Side, Font, Alignment

# Create a new workbook and select the active worksheet
output_wb = openpyxl.Workbook()
bs_ws = output_wb.active
bs_ws.title = "BS Notes"

# Create another worksheet for PL notes
pl_ws = output_wb.create_sheet(title="PL Notes")

# Company details
company_name = "Saptaranga Research and Organic Private Limited"
address_line1 = "Plot No 45,"
address_line2 = "Ravindra Nagar P.M.G. Society"
address_line3 = "NOTES ANNEXED TO AND FORMING PART OF ACCOUNTS FOR THE YEAR ENDING 31-Mar-2023"
dateCurr = "As on 31-Mar-2023"
datePrev = "As on 31-Mar-2022"

# Function to set the title and address in a worksheet
def set_title_and_address(ws):
    ws.merge_cells('C1:H1')
    ws['C1'] = company_name
    ws.merge_cells('C2:H2')
    ws['C2'] = address_line1
    ws.merge_cells('C3:H3')
    ws['C3'] = address_line2
    ws.merge_cells('C4:H4')
    ws['C4'] = address_line3

    title_font = Font(size=14, bold=True)
    for cell in ['C1', 'C2', 'C3', 'C4']:
        ws[cell].font = title_font
        ws[cell].alignment = Alignment(horizontal='center')

    ws.column_dimensions['C'].width = 50
    ws.column_dimensions['G'].width = 15
    ws.column_dimensions['H'].width = 15

# Apply title and address to both sheets
set_title_and_address(bs_ws)
set_title_and_address(pl_ws)

# Define the border styles
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

thick_border = Border(
    left=Side(style='thick'),
    right=Side(style='thick'),
    top=Side(style='thick'),
    bottom=Side(style='thick')
)

# Function to add sections to the specified worksheet
def add_section(headers, search_head, note, category, row_offset):
    # Select the appropriate worksheet based on category
    output_ws = bs_ws if category == 'BS' else pl_ws

    # Load the input Excel file
    input_wb = openpyxl.load_workbook("C:\\Users\\jadit\\OneDrive\\Desktop\\BalanceSheetProject\\ExcelFiles\\InputForScheduleTemplate\\InputForASchedule.xlsx")
    input_ws = input_wb.active

    # Set the headers
    output_ws[f'C{6 + row_offset}'] = note
    output_ws[f'G{5 + row_offset}'] = "In Rs. hundreds"
    output_ws[f'C{7 + row_offset + len(headers)}'] = "Total"

    output_ws[f'G{6 + row_offset}'] = dateCurr
    output_ws[f'H{6 + row_offset}'] = datePrev

    header_font = Font(size=10, bold=True)
    output_ws[f'G{5 + row_offset}'].font = header_font
    output_ws[f'G{6 + row_offset}'].font = header_font
    output_ws[f'H{6 + row_offset}'].font = header_font
    output_ws[f'C{6 + row_offset}'].font = header_font
    output_ws[f'C{7 + row_offset + len(headers)}'].font = header_font

    # Apply the border to the header cells
    for cell in [f'C{6 + row_offset}', f'G{5 + row_offset}', f'G{6 + row_offset}', f'H{6 + row_offset}']:
        output_ws[cell].border = thin_border

    # Place each header in a separate row
    for i, header in enumerate(headers):
        output_ws[f'C{7 + row_offset + i}'] = header
        output_ws[f'C{7 + row_offset + i}'].border = thin_border

    # Specify the header name of the column you want to iterate through
    header_name = "Major Head"
    subHeadList = []
    mappedDict = {}

    # Find the index of the column with the specified header name
    column_index = None
    for col in range(1, input_ws.max_column + 1):
        if input_ws.cell(row=1, column=col).value == header_name:
            column_index = col
            break

    if column_index is not None:
        # Iterate through each row in the column
        for row in range(2, input_ws.max_row + 1):  # Start from 2 to skip the header row
            cell_value = input_ws.cell(row=row, column=column_index).value
            if cell_value == search_head:
                subHeadList.append(input_ws.cell(row=row, column=column_index + 1).value)

        for row in range(2, input_ws.max_row + 1):
            sub_head_value = input_ws.cell(row=row, column=column_index + 1).value
            if sub_head_value in subHeadList:
                mappedDict[sub_head_value] = [
                    input_ws.cell(row=row, column=column_index - 2).value,
                    input_ws.cell(row=row, column=column_index - 1).value
                ]

    sumOfcurr = sum(mappedDict[key][0] for key in mappedDict if mappedDict[key][0] is not None)
    sumofprev = sum(mappedDict[key][1] for key in mappedDict if mappedDict[key][1] is not None)

    for c in range(1, output_ws.max_column + 1):
        for r in range(1, output_ws.max_row + 1):
            cell_value = output_ws.cell(row=r, column=c).value
            if cell_value in mappedDict:
                output_ws.cell(row=r, column=c + 4).value = mappedDict[cell_value][0]
                output_ws.cell(row=r, column=c + 5).value = mappedDict[cell_value][1]
            if cell_value == "Total" and (r >= (7 + row_offset) and r <= (7 + row_offset + len(headers))):
                output_ws.cell(row=r, column=c + 4).value = sumOfcurr
                output_ws.cell(row=r, column=c + 5).value = sumofprev

    # Apply thick border to the row immediately after the section
    for col in range(3, 9):  # C to H
        cell = output_ws.cell(row=7 + row_offset + len(headers) , column=col)
        cell.border = thin_border
    cell1=output_ws[f'G{5 + row_offset}']
    cell1.border=None
    input_wb.close()

# Define the sections to be added along with their categories
otherCurrentLiabilities = [
    "Current Maturities Of Finance Lease Obligations",
    "Interest Accrued But Not Due On Borrowings",
    "Interest Accrued And Due On Borrowings",
    "Income Received In Advance",
    "Unpaid Dividends",
    "Application Money Received For Allotment Of Securities And Due For Refund And Interest Accrued Thereon",
    "Unpaid Matured Deposits And Interest Accrued Thereon",
    "Unpaid Matured Debentures And Interest Accrued Thereon",
    "Other Payables (Specify Nature)"
]
DeferredTaxLiability = ["Deferred Tax Liability"]
longtermliabilities = ["Trade Payables", "Others"]
cashandcashequivalent = [
    "Cheques, Drafts On Hand",
    "Cash On Hand",
    "Cash At Bank",
    "Other C & CE (Specify Nature)"
]
CurrentInvestments = [
    "Investments In Equity Instruments",
    "Investments In Preference Shares",
    "Investments In Government Or Trust Securities",
    "Investments In Debentures Or Bonds",
    "Investments In Mutual Funds",
    "Investments In Partnership Firms",
    "Other Investments (Specify Nature)"
]
LongtermProvisions = [
    "Provision For Employee Benefits",
    "Others Long Term Provision (Specify Nature)"
]
shortTermProvisions = [
    "Provision for Employees Benefits",
    "Provision for Income Tax",
    "Other Short Term Provision (Specify Nature)"
]
nonCurrentInvestments = [
    "Investment Property",
    "Investments In Equity Instruments",
    "Investments In Preference Shares",
    "Investments In Government Or Trust Securities",
    "Investments In Debentures Or Bonds",
    "Investments In Mutual Funds",
    "Investments In Partnership Firms",
    "Other Non-Current Investments (Specify Nature)"
]
deferredtaxassets = ["Deferred Tax Asset"]
inventories = [
    "Raw Materials",
    "Work-In-Progress",
    "Finished Goods",
    "Stock-In-Trade",
    "Stores And Spares",
    "Loose Tools",
    "Other Inventory (Specify Nature)"
]
othercurrentAssets=[
    "Other Current Assets 1",
    "Other Current Assets 2",
    "Other Current Assets 3",
    "Other Current Assets 4",
    "Other Current Assets 5"
]
revenueFromOperations=[
    "Sale Of Product",
    "Sale of Services",
    "Grants or Donations Received",
    "Other Operating Revenue",
    "Less: Excise Duty"
]
otherIncome=[
    "Interest Income",
    "Dividend Income",
    "Net Gain/Loss On Sale Of Investments",
    "Other Non - Operating Income"
]
purchaseofstockintrade=["Purchases"]
sections = [
    {"header": DeferredTaxLiability, "search_head": "Deferred Tax Liability", "note": "Note : Deferred Tax Liability", "category": "BS"},
    {"header": longtermliabilities, "search_head": "Other Long-term Liabilities", "note": "Note : Other Long-term Liabilities", "category": "BS"},
    {"header": LongtermProvisions, "search_head": "Long-Term Provisions", "note": "Note : Long-Term Provisions", "category": "BS"},
    {"header": otherCurrentLiabilities, "search_head": "Other Current Liabilities", "note": "Note : Other Current Liabilities", "category": "BS"},    
    {"header": shortTermProvisions, "search_head": "Short Term Provisions", "note": "Note : Short Term Provisions", "category": "BS"},
    {"header": nonCurrentInvestments, "search_head": "Non Current Investments", "note": "Note : Non Current Investments", "category": "BS"},
    {"header": deferredtaxassets, "search_head": "Deferred Tax Asset", "note": "Note : Deferred Tax Asset", "category": "BS"},
    {"header": CurrentInvestments, "search_head": "Current Investments", "note": "Note : Current Investments", "category": "BS"},
    {"header": inventories, "search_head": "Inventories", "note": "Note : Inventories", "category": "BS"},
    {"header": cashandcashequivalent, "search_head": "Cash and Cash Equivalents", "note": "Note : Cash and Cash Equivalents", "category": "BS"},
    {"header": othercurrentAssets, "search_head": "Other Current Assets", "note": "Note : Other Current Assets", "category": "BS"},
    {"header": revenueFromOperations, "search_head": "Revenue From Operations", "note": "Note : Revenue From Operations", "category": "PL"},
    {"header": otherIncome, "search_head": "Other Income", "note": "Note : Other Income", "category": "PL"},
    {"header": purchaseofstockintrade, "search_head": "Purchase Of Stock in Trade", "note": "Note : Purchase Of Stock in Trade", "category": "PL"}

]

# Apply each section
row_offset_bs = 0  # Row offset for BS sheet
row_offset_pl = 0  # Row offset for PL sheet

for section in sections:
    if section["category"] == "BS":
        add_section(section["header"], section["search_head"], section["note"], section["category"], row_offset_bs)
        row_offset_bs += len(section["header"]) + 3  # Adjust row offset for next BS section
    else:
        add_section(section["header"], section["search_head"], section["note"], section["category"], row_offset_pl)
        row_offset_pl += len(section["header"]) + 3  # Adjust row offset for next PL section

# Function to apply border to the area encompassing first and last non-null values for both worksheets
def apply_border_to_first_last_non_null(ws):
    first_non_null_row = None
    first_non_null_col = None
    last_non_null_row = None
    last_non_null_col = None

    for row in range(1, ws.max_row + 1):
        for col in range(1, ws.max_column + 1):
            cell_value = ws.cell(row=row, column=col).value
            if cell_value is not None:
                if first_non_null_row is None:
                    first_non_null_row = row
                    first_non_null_col = col
                last_non_null_row = row
                last_non_null_col = col

    if first_non_null_row is not None and first_non_null_col is not None:
        # Apply border to the area encompassing first and last non-null values
        for row in range(first_non_null_row, last_non_null_row + 1):
            for col in range(first_non_null_col, last_non_null_col + 1):
                ws.cell(row=row, column=col).border = thin_border

# Apply border to the area encompassing first and last non-null values for both worksheets
apply_border_to_first_last_non_null(bs_ws)
apply_border_to_first_last_non_null(pl_ws)

# Save the output workbook
output_wb.save("C:\\Users\\jadit\\OneDrive\\Desktop\\BalanceSheetProject\\GeneratedOutput\\TemplateOutput\\Schedules-distributed-BS-PL-Test.xlsx")
